# CAPTSTONE PROJECT 3:

### DATA CLEANING

***

##### CLEAN DATA:
* Import .csv file in local Data Folder as DataFrame: df
* Examine the contents of DataFrame and address any issues regard NaN or others empty values
* Save updated dataFrame as df_cleaned.csv

In [1]:
# Import necessary modules for DataFrame cleaning
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import finnhub